# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,old town,50.0870,14.4202,6.37,74,40,6.69,CZ,1706310711
1,1,boguchany,58.3814,97.4531,-22.30,100,95,0.89,RU,1706310711
2,2,port-aux-francais,-49.3500,70.2167,6.71,95,81,14.95,TF,1706310711
3,3,karratha,-20.7377,116.8463,25.50,77,7,6.36,AU,1706310520
4,4,san jose,37.3394,-121.8950,18.20,57,100,2.06,US,1706310510


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",
                                         frame_width = 600, frame_height = 500, 
                                          size = "Humidity", color = "City")

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_temp = city_data_df.loc[(city_data_df["Max Temp"].between(21, 27)), :]
narrow_wind = narrow_temp.loc[narrow_temp["Wind Speed"] < 4.5]
narrow_clouds = narrow_wind.loc[narrow_wind["Cloudiness"] < 15]

# Drop any rows with null values
narrow_fin = narrow_clouds.dropna()

# Display sample data
narrow_fin

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,port saint john's,-31.6229,29.5448,23.95,88,11,3.47,ZA,1706310716
22,22,guichon,-32.3500,-57.2000,23.84,46,0,3.19,UY,1706310719
74,74,amagunze,6.3267,7.6539,25.61,21,12,2.32,NG,1706310737
122,122,axim,4.8699,-2.2405,26.57,84,3,2.02,GH,1706310754
137,137,trevelin,-43.0858,-71.4639,23.84,37,2,4.07,AR,1706310759
174,174,beira,-19.8436,34.8389,24.62,82,0,2.66,MZ,1706310772
238,238,kita,13.0349,-9.4895,23.52,15,0,2.74,ML,1706310795
306,306,tazacorte,28.6290,-17.9293,21.75,27,0,2.57,ES,1706310818
309,309,orangeburg,33.4918,-80.8557,23.23,72,0,2.57,US,1706310819
343,343,geraldton,-28.7667,114.6000,22.23,64,0,2.57,AU,1706310552


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_fin[["City", "Country", "Lat", "Lng", "Humidity"]].copy().reset_index().drop(columns = "index")

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port saint john's,ZA,-31.6229,29.5448,88,
1,guichon,UY,-32.3500,-57.2000,46,
2,amagunze,NG,6.3267,7.6539,21,
3,axim,GH,4.8699,-2.2405,84,
4,trevelin,AR,-43.0858,-71.4639,37,
5,beira,MZ,-19.8436,34.8389,82,
6,kita,ML,13.0349,-9.4895,15,
7,tazacorte,ES,28.6290,-17.9293,27,
8,orangeburg,US,33.4918,-80.8557,72,
9,geraldton,AU,-28.7667,114.6000,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE